In [15]:
import wechrom


Initialize the nucleosome system with a pdbx file we prepared with 147 bp wrapped DNA, 38 bp linker DNA on each end and a histone core particle. This cif file also includes two virtual sites at the two ends for external force illustration. 

In [5]:
nuc_223bp = wechrom.SingleNucleoSystem("singleN_L38_endvs.cif", verbose = True)

Apply default forces designed for wechrom

In [6]:
nuc_223bp.addDefaultForces()

Building connectivity terms...... done
Building excluded volume force...... done
Preparing the associative memory files...... done
Building DNA intra-strand associative memory force...... done
Building DNA inter-strand associative memory force...... done
Preparing the nucleosome associative memory files...... done
Building nucleosome center associative memory force...... done
Building nucleosome neighbor associative memory force...... done


Here we provide an example of applying external force to the system. The desired external force is to pull the two ends apart in x direction. You may use any force supported by openmm

In [8]:
from openmm import CustomExternalForce

def stretch_term(we, k_stretching=2.0, forceDirect="x", appliedSite=0):
    stretching = CustomExternalForce(f"({k_stretching})*({forceDirect})")
    stretching.addParticle(we.virtualSites[appliedSite])
    return stretching

nuc_223bp.addForce(stretch_term(nuc_223bp, forceDirect="x", appliedSite=0), "pull_h")
nuc_223bp.addForce(stretch_term(nuc_223bp, forceDirect="-x", appliedSite=-1), "pull_t")

Initialize the simulation with default integrator and wechrom topology

In [9]:
nuc_223bp.initializeSimulation(platform='CUDA')

Langevin integrator and simulation initialized


Here we provide an example of adding a pdb reporter to the simulation

In [13]:
from openmm.app import PDBReporter # openmm version >= 7.6
# from simtk.openmm.app import PDBReporter # openmm version < 7.6
steps = 1e3
reportFreq = 1e1
nuc_223bp.simulation.reporters.append(PDBReporter("./movie.pdb", reportFreq))

Run the simulation

In [14]:
nuc_223bp.runSteps(steps = steps, reportFreq = reportFreq)

Simulation will take 1000 steps and get reported every 10 steps
----------------Simulation Starts----------------


100%|██████████| 100/100 [00:02<00:00, 38.37it/s]


Simulation done.
Please check your trajectory file movie.dcd, energy file energy.txt at your output directory d:\wechrom\wechrom_private\examples\nucleosome_223bp
